In [1]:
pip install sklite

In [2]:
import json

# Open and read the JSON file
with open('1stfloorwifi.json', 'r') as file:
    floor1 = json.load(file)
with open('2ndfloorwifi.json', 'r') as file:
    floor2 = json.load(file)
with open('3rdfloorwifi.json', 'r') as file:
    floor3 = json.load(file)
with open('4thfloorwifi.json', 'r') as file:
    floor4 = json.load(file)
with open('5thfloorwifi.json', 'r') as file:
    floor5 = json.load(file)
with open('terracewifi.json', 'r') as file:
    terrace = json.load(file)

data = floor1 + floor2 + floor3 + floor4 + floor5 + terrace

In [4]:
import csv

# Collect all unique BSSIDs from the data
bssids = set()
for entry in data:
    for wifi in entry['wifi']:
        bssids.add(wifi['bssid'])

sort_bssid = sorted(bssids)  # Sort BSSIDs for consistent column ordering
print(sort_bssid)

['0a:7b:c8:a8:9d:d5', '0a:7b:c8:a9:4d:96', '0a:7b:c8:a9:4e:dc', '0a:7b:c8:a9:4f:8a', '0a:7b:c8:a9:50:a6', '0a:7b:c8:a9:50:e2', '0a:7b:c8:a9:52:b5', '0a:7b:c8:a9:57:2c', '0a:7b:c8:a9:5c:b2', '0a:7b:c8:a9:5f:e7', '0a:7b:c8:a9:61:fd', '0a:7b:c8:a9:62:ba', '0a:7b:c8:a9:64:39', '0a:7b:c8:a9:64:c7', '0a:7b:c8:a9:6a:62', '0a:7b:c8:ab:04:1e', '0a:7b:d8:a8:9d:d5', '0a:7b:d8:a9:4d:96', '0a:7b:d8:a9:4e:dc', '0a:7b:d8:a9:4f:8a', '0a:7b:d8:a9:50:a6', '0a:7b:d8:a9:50:e2', '0a:7b:d8:a9:52:b5', '0a:7b:d8:a9:57:2c', '0a:7b:d8:a9:5c:b2', '0a:7b:d8:a9:5f:e7', '0a:7b:d8:a9:61:fd', '0a:7b:d8:a9:62:ba', '0a:7b:d8:a9:64:39', '0a:7b:d8:a9:64:c7', '0a:7b:d8:a9:6a:62', '0a:7b:d8:ab:04:1e', '0a:7b:d8:ab:1d:1b', 'e2:55:a8:10:8f:51', 'e2:55:a8:10:8f:83', 'e2:55:a8:10:8f:9f', 'e2:55:a8:10:8f:b7', 'e2:55:a8:10:96:5d', 'e2:55:a8:10:96:88', 'e2:55:a8:10:a9:b4', 'e2:55:a8:10:af:1e', 'e2:55:a8:10:af:1f', 'e2:55:a8:10:af:21', 'e2:55:a8:10:af:3a', 'e2:55:a8:10:af:4b', 'e2:55:a8:10:e5:48', 'e2:55:a8:10:e5:4a', 'e2:55:a8:10

In [5]:
output_file = 'wifidatatrain.csv'

# Collect all unique BSSIDs from the data
bssids = set()
for entry in data:
    for wifi in entry['wifi']:
        bssids.add(wifi['bssid'])

bssids = sort_bssid

# Create the CSV file
with open(output_file, 'w', newline='') as csvfile:
    # Create CSV writer object
    csvwriter = csv.writer(csvfile)

    # Write the header (BSSID values)
    header = ['index','room'] + bssids
    csvwriter.writerow(header)

    # Write rows for each data entry
    for index, entry in enumerate(data):
        row = [
            index,  # Index
            entry['room'],  # Room
            #entry['place_collection_number']  # Place Collection Number
        ]
        bssid_signal = {wifi['bssid']: wifi['signal_strength'] for wifi in entry['wifi']}

        # Fill the row with signal strength, or leave blank if no signal for that BSSID
        row.extend([bssid_signal.get(bssid, 0) for bssid in bssids])
        csvwriter.writerow(row)

print(f"Data has been successfully written to {output_file}")

Data has been successfully written to wifidatatrain.csv


In [6]:
import pandas as pd

# Load the uploaded CSV file to examine its contents
file_path = 'wifidatatrain.csv'
data = pd.read_csv(file_path)

signal_columns = data.columns.difference(['index', 'room'])

def transform_row(row):
    # Extract only signal values for processing
    signals = row[signal_columns]

    # Find the highest non-zero signal value
    min_non_zero = signals[signals != 0].min()
    max_non_zero = signals[signals != 0].max()

    if pd.isna(min_non_zero) or min_non_zero == 0:  # Handle rows with all zeros
        return row

    range = max_non_zero - min_non_zero

    # Calculate the transformation for non-zero values that are not the highest value
    transformed_signals = signals.apply(
        #lambda x: x / max_non_zero if x == max_non_zero else -(max_non_zero - x) / max_non_zero if x != 0 else x
        lambda x: 0 if x == min_non_zero else -(min_non_zero - x) / range if x != 0 else -1
    )

    # Update the row with the transformed signal values
    row[signal_columns] = transformed_signals
    return row

# Apply the transformation to each row again
transformed_data = data.apply(transform_row, axis=1)

# Save the modified DataFrame to a new file
output_path = 'wifidata_transformed.csv'
transformed_data.to_csv(output_path, index=False)

output_path

'wifidata_transformed.csv'

In [7]:
from sklearn.preprocessing import LabelEncoder

le_area = LabelEncoder()

train = pd.read_csv("wifidata_transformed.csv")

train['room_n'] = le_area.fit_transform(train['room'])

target = train["room_n"]
signal = train.drop(["index","room", "room_n"],axis="columns")

In [8]:
from itertools import groupby
a = -1
b = []
numbers = train["room_n"]
rooms = train["room"]
for x in range(len(numbers)):
  if numbers[x] != a:
    a = numbers[x]
    b.append((numbers[x],rooms[x]))
  else:
    continue
b.sort(key=lambda x: x[0])
res = {key: [value for key, value in group] for key, group in groupby(b, key=lambda x: x[0])}
for room in b:
  print(room)

(0, '3005')
(1, '3015')
(2, '3rd floor_1')
(3, '3rd floor_10')
(4, '3rd floor_12')
(5, '4th floor_11')
(6, '4th floor_12')
(7, '4th floor_2')
(8, '4th floor_library')
(9, 'JOMA Cafe')
(10, 'Senior Lounge')
(11, 'Terrace')
(12, 'outside 5007')
(13, 'outside 5008')
(14, 'outside 5009')
(15, 'outside 5th floor conference room')
(16, 'outside BlackBox')
(17, 'outside gym')
(18, 'outside room 3010')
(19, 'outside room 3011')
(20, 'outside room 3012')
(21, 'outside room 3013')
(22, 'outside room 4010')
(23, 'outside room 4011')
(24, 'outside room 4012')
(25, 'room 3008')
(26, 'room 3009')
(27, 'room 3010')
(28, 'room 3013')
(29, 'room 4008')


In [9]:
from sklite import LazyExport
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=28)
model.fit(signal, target)

lazy = LazyExport(model)
lazy.save('wififlex.json')

/usr/local/lib/python3.11/dist-packages/sklite/lib/abstract.py:85: UserWarning: /root/.sklite has been created.
  warnings.warn(wrn)
